In [26]:
import numpy as np
import pandas as pd
from itertools import product
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import random
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices()

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# List all physical devices
gpus = tf.config.list_physical_devices('GPU')
print("Available GPUs:")
for gpu in gpus:
    print(f" - {gpu.name}")

Available GPUs:
 - /physical_device:GPU:0


In [16]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled for GPUs.")
    except RuntimeError as e:
        print(e)

Memory growth enabled for GPUs.


In [8]:
traffic_data = pd.read_csv('../Traffic data/SCOOT/data_470_hourly/flows/GD030A_S.csv')

## 1. Recover timestamp

In [9]:
# Define the recover_timestamp function
def recover_timestamp(data):
    # Combine 'date' and 'time' to form a datetime column
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'].astype(str) + ':00', format='%Y-%m-%d %H:%M')

    # Set 'datetime' as index
    data = data.set_index('datetime')

    # Create a complete range of timestamps with hourly frequency
    full_time_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')

    # Reindex the data to include all timestamps, filling missing rows with NaN
    data_full = data.reindex(full_time_range)

    return data_full

In [10]:
# Apply the recover_timestamp function to recover the full time series
traffic_full = recover_timestamp(traffic_data)
traffic_full

,date,time,flow
2019-10-01 00:00:00,2019-10-01,0.0,15.0
2019-10-01 01:00:00,2019-10-01,1.0,9.0
2019-10-01 02:00:00,2019-10-01,2.0,9.0
2019-10-01 03:00:00,2019-10-01,3.0,7.0
2019-10-01 04:00:00,2019-10-01,4.0,9.0
...,...,...,...
2023-09-30 19:00:00,2023-09-30,19.0,129.0
2023-09-30 20:00:00,2023-09-30,20.0,119.0
2023-09-30 21:00:00,2023-09-30,21.0,106.0
2023-09-30 22:00:00,2023-09-30,22.0,88.0


## 2. Train, validate, test data split

In [11]:
# train_set = traffic_full[:'2022-02-28 23:00:00']
# valid_set = traffic_full['2022-03-01 00:00:00':'2022-12-31 23:00:00']
# test_set = traffic_full['2023-01-01 00:00:00':]
train_set = traffic_full['2022-06-03 00:00:00':'2023-03-31 23:00:00']
valid_set = traffic_full['2023-04-01 00:00:00':'2023-06-30 23:00:00']
test_set = traffic_full['2023-07-01 00:00:00':]
print('Proportion of train_set : {:.4f}'.format(len(train_set)/len(traffic_full['2022-06-03 00:00:00':])))
print('Proportion of valid_set : {:.4f}'.format(len(valid_set)/len(traffic_full['2022-06-03 00:00:00':])))
print('Proportion of test_set : {:.4f}'.format(len(test_set)/len(traffic_full['2022-06-03 00:00:00':])))

Proportion of train_set : 0.6227
Proportion of valid_set : 0.1876
Proportion of test_set : 0.1897


In [12]:
print(train_set.isnull().sum(), len(train_set))
print(valid_set.isnull().sum(),len(valid_set))
print(test_set.isnull().sum(),len(test_set))

date    16
time    16
flow    16
dtype: int64 7248
date    61
time    61
flow    61
dtype: int64 2184
date    342
time    342
flow    342
dtype: int64 2208


## 3. Normalise the data 

In [13]:
# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data's 'flow' feature
scaler.fit(train_set[['flow']])

# Transform the 'flow' feature in all datasets
train_set.loc[:, 'flow_scaled'] = scaler.transform(train_set[['flow']])
valid_set.loc[:, 'flow_scaled'] = scaler.transform(valid_set[['flow']])
test_set.loc[:, 'flow_scaled'] = scaler.transform(test_set[['flow']])

C:\Users\Amanda\AppData\Local\Temp\ipykernel_22452\1100395100.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set.loc[:, 'flow_scaled'] = scaler.transform(train_set[['flow']])
C:\Users\Amanda\AppData\Local\Temp\ipykernel_22452\1100395100.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_set.loc[:, 'flow_scaled'] = scaler.transform(valid_set[['flow']])
C:\Users\Amanda\AppData\Local\Temp\ipykernel_22452\1100395100.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

## 4. Split the data into X and y

In [14]:
def create_sequences(data, input_length, forecast_horizon):
    """
    Creates input-output sequences for time series data, excluding any sequences containing NaN values.
    
    Parameters:
    - data: pandas DataFrame containing the data. Must include the 'flow_scaled' column.
    - input_length: int, number of past time steps to include in each input sequence.
    - forecast_horizon: int, number of future steps to predict.
    
    Returns:
    - X: numpy array of shape (num_valid_samples, input_length, num_features)
    - y: numpy array of shape (num_valid_samples, forecast_horizon)
    """
    X, y = [], []
    num_features = data.shape[1]
    total_length = input_length + forecast_horizon
    
    for i in range(input_length, len(data) - forecast_horizon + 1):
        # Extract the input sequence
        X_seq = data.iloc[i - input_length:i]['flow_scaled'].values
        # Extract the target sequence
        y_seq = data.iloc[i:i + forecast_horizon]['flow_scaled'].values
        
        # Check for NaN values in the input sequence and target sequence
        if not np.isnan(X_seq).any() and not np.isnan(y_seq).any():
            X.append(X_seq)
            y.append(y_seq)
        else:
            # Optionally, log or count the skipped sequences
            pass  # Simply skip sequences with NaNs
        
    # Convert to numpy arrays and reshape X to match LSTM expected input (samples, timesteps, features)
    X = np.array(X).reshape(-1, input_length, 1)
    y = np.array(y).reshape(-1, forecast_horizon)
    
    return X, y

## 5. Create X and y

#### We will use
* the last 24*N steps

*  to forecast current (0 step) and 5 steps ahead

In [15]:
# Define Input Sequence Lengths
input_lengths = [24 * i for i in range(1, 22)]  # [24, 48, ..., 168]

In [16]:
from collections import defaultdict
data_dict = defaultdict(dict)

for length in input_lengths:
    print(f"Processing input length: {length}")
    
    # Create sequences with forecast_horizon=6
    X_train, y_train = create_sequences(train_set, length, forecast_horizon=6)
    X_val, y_val = create_sequences(valid_set, length, forecast_horizon=6)
    X_test, y_test = create_sequences(test_set, length, forecast_horizon=6)
    
    # Store in the dictionary
    data_dict[length]['X_train'] = X_train
    data_dict[length]['y_train'] = y_train
    data_dict[length]['X_val'] = X_val
    data_dict[length]['y_val'] = y_val
    data_dict[length]['X_test'] = X_test
    data_dict[length]['y_test'] = y_test
    
    # Print shapes and ensure no NaNs
    print(f"  X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
    print(f"  X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
    print(f"  X_test shape: {X_test.shape}, y_test shape: {y_test.shape}\n")

Processing input length: 24
  X_train shape: (7145, 24, 1), y_train shape: (7145, 6)
  X_val shape: (2007, 24, 1), y_val shape: (2007, 6)
  X_test shape: (1719, 24, 1), y_test shape: (1719, 6)

Processing input length: 48
  X_train shape: (7073, 48, 1), y_train shape: (7073, 6)
  X_val shape: (1911, 48, 1), y_val shape: (1911, 6)
  X_test shape: (1604, 48, 1), y_test shape: (1604, 6)

Processing input length: 72
  X_train shape: (7001, 72, 1), y_train shape: (7001, 6)
  X_val shape: (1815, 72, 1), y_val shape: (1815, 6)
  X_test shape: (1508, 72, 1), y_test shape: (1508, 6)

Processing input length: 96
  X_train shape: (6929, 96, 1), y_train shape: (6929, 6)
  X_val shape: (1719, 96, 1), y_val shape: (1719, 6)
  X_test shape: (1412, 96, 1), y_test shape: (1412, 6)

Processing input length: 120
  X_train shape: (6857, 120, 1), y_train shape: (6857, 6)
  X_val shape: (1623, 120, 1), y_val shape: (1623, 6)
  X_test shape: (1316, 120, 1), y_test shape: (1316, 6)

Processing input length: 1

## 6. Build LSTM model

In [17]:
def build_lstm_model(hyperparams, input_length):
    
    model = Sequential()
    model.add(LSTM(units=hyperparams['units'], activation='relu', input_shape=(input_length, 1)))
    model.add(Dropout(rate=hyperparams['dropout']))
    model.add(Dense(6))  # Output layer for multi-step forecasting

    # Compile the model with MSE as the loss function
    optimizer = keras.optimizers.Adam(learning_rate=hyperparams['learning_rate'])
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
    
    return model

## 7. Define the hyperparameter grid

In [13]:
# Hyperparameter options
units_list = [32, 64, 128, 256]
dropout_rates = [0.0, 0.1, 0.2, 0.3, 0.4]
learning_rates = [0.01, 0.001, 0.0005]
batch_sizes = [32, 64, 128]

# Create all possible combinations
all_combinations = list(product(units_list, dropout_rates, learning_rates, batch_sizes))

In [14]:
all_combinations

[(32, 0.0, 0.01, 32),
 (32, 0.0, 0.01, 64),
 (32, 0.0, 0.01, 128),
 (32, 0.0, 0.001, 32),
 (32, 0.0, 0.001, 64),
 (32, 0.0, 0.001, 128),
 (32, 0.0, 0.0005, 32),
 (32, 0.0, 0.0005, 64),
 (32, 0.0, 0.0005, 128),
 (32, 0.1, 0.01, 32),
 (32, 0.1, 0.01, 64),
 (32, 0.1, 0.01, 128),
 (32, 0.1, 0.001, 32),
 (32, 0.1, 0.001, 64),
 (32, 0.1, 0.001, 128),
 (32, 0.1, 0.0005, 32),
 (32, 0.1, 0.0005, 64),
 (32, 0.1, 0.0005, 128),
 (32, 0.2, 0.01, 32),
 (32, 0.2, 0.01, 64),
 (32, 0.2, 0.01, 128),
 (32, 0.2, 0.001, 32),
 (32, 0.2, 0.001, 64),
 (32, 0.2, 0.001, 128),
 (32, 0.2, 0.0005, 32),
 (32, 0.2, 0.0005, 64),
 (32, 0.2, 0.0005, 128),
 (32, 0.3, 0.01, 32),
 (32, 0.3, 0.01, 64),
 (32, 0.3, 0.01, 128),
 (32, 0.3, 0.001, 32),
 (32, 0.3, 0.001, 64),
 (32, 0.3, 0.001, 128),
 (32, 0.3, 0.0005, 32),
 (32, 0.3, 0.0005, 64),
 (32, 0.3, 0.0005, 128),
 (32, 0.4, 0.01, 32),
 (32, 0.4, 0.01, 64),
 (32, 0.4, 0.01, 128),
 (32, 0.4, 0.001, 32),
 (32, 0.4, 0.001, 64),
 (32, 0.4, 0.001, 128),
 (32, 0.4, 0.0005, 32),

## 8. Defining manual grid search

In [15]:
from tensorflow.keras import models, layers, optimizers

In [18]:
# Initialize a list to store the results
results = []

for length in input_lengths:
    print(f"Starting grid search for input length: {length}")
    
    X_train = data_dict[length]['X_train']
    y_train = data_dict[length]['y_train']
    X_val = data_dict[length]['X_val']
    y_val = data_dict[length]['y_val']
    
    best_mse = float('inf')
    best_params = {}
    best_model = None
    
    for idx, combination in enumerate(all_combinations):
        # Extract hyperparameters
        hyperparams = {
            'units': combination[0],
            'dropout': combination[1],
            'learning_rate': combination[2],
            'batch_size': combination[3]
        }
        
        print(f"  Evaluating combination {idx + 1}/{len(all_combinations)}: {hyperparams}")
        
        model = build_lstm_model(hyperparams, length)
        
        # Early Stopping Callback
        early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True,verbose=1)
        
        # Train the model
        history = model.fit(
            X_train, y_train,
            epochs=50,
            batch_size=hyperparams['batch_size'],
            validation_data=(X_val, y_val),
            callbacks=[early_stop],
            verbose=0  # Set to 1 to see training progress
        )
        
        # Retrieve the best validation MSE from the history
        current_best_mse = min(history.history['val_loss'])
        print(f"Validation loss: {current_best_mse:.4f}")
        
        # Check if this is the best model so far
        if current_best_mse < best_mse:
            best_mse = current_best_mse
            best_params = hyperparams.copy()
            best_model = model  # Optionally, save the model if needed
    
    # After evaluating all combinations, store the best results
    results.append({
        'Input_Length': length,
        'Best_MSE': best_mse,
        'Validation_MSE': mean_squared_error(y_val, best_model.predict(X_val)),
        'Validation_MAE': mean_absolute_error(y_val, best_model.predict(X_val)),
        'Validation_MAPE': mean_absolute_percentage_error(y_val, best_model.predict(X_val)) * 100,  # In percentage
        'Validation_RMSE': np.sqrt(mean_squared_error(y_val, best_model.predict(X_val))),
        'Best_Hyperparameters': best_params
    })
    
    print(f"Completed grid search for input length: {length}")
    print(f"  Best Validation MSE: {best_mse:.4f}")
    print(f"  Best Hyperparameters: {best_params}\n")

Starting grid search for input length: 360
  Evaluating combination 1/180: {'units': 32, 'dropout': 0.0, 'learning_rate': 0.01, 'batch_size': 32}


/home/yl2672496l/Yue/code/jupyter_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1729247188.981079 2088033 service.cc:146] XLA service 0x74a0800b7a60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729247188.981111 2088033 service.cc:154]   StreamExecutor device (0): NVIDIA A30, Compute Capability 8.0
2024-10-18 11:26:29.014304: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-18 11:26:29.408747: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
I0000 00:00:1729247190.328027 2088033 device_compiler.h:188] Compiled cluster using XLA!  This lin

Epoch 22: early stopping
Restoring model weights from the end of the best epoch: 12.
Validation loss: 0.0038
  Evaluating combination 2/180: {'units': 32, 'dropout': 0.0, 'learning_rate': 0.01, 'batch_size': 64}
Epoch 38: early stopping
Restoring model weights from the end of the best epoch: 28.
Validation loss: 0.0037
  Evaluating combination 3/180: {'units': 32, 'dropout': 0.0, 'learning_rate': 0.01, 'batch_size': 128}
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation loss: nan
  Evaluating combination 4/180: {'units': 32, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 32}
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation loss: nan
  Evaluating combination 5/180: {'units': 32, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 64}
Epoch 43: early stopping
Restoring model weights from the end of the best epoch: 33.
Validation loss: 0.0038
  Evaluating combination 6/180: {'units': 32, 'd

2024-10-18 15:04:04.793294: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_19', 40 bytes spill stores, 40 bytes spill loads



Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation loss: nan
  Evaluating combination 92/180: {'units': 128, 'dropout': 0.0, 'learning_rate': 0.01, 'batch_size': 64}
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation loss: nan
  Evaluating combination 93/180: {'units': 128, 'dropout': 0.0, 'learning_rate': 0.01, 'batch_size': 128}
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation loss: nan
  Evaluating combination 94/180: {'units': 128, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 32}
Epoch 34: early stopping
Restoring model weights from the end of the best epoch: 24.
Validation loss: 0.0039
  Evaluating combination 95/180: {'units': 128, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 64}
Epoch 34: early stopping
Restoring model weights from the end of the best epoch: 24.
Validation loss: 0.0038
  Evaluating combination 96/180: {'units': 1

...

Epoch 30: early stopping
Restoring model weights from the end of the best epoch: 20.
Validation loss: 0.0051
  Evaluating combination 2/180: {'units': 32, 'dropout': 0.0, 'learning_rate': 0.01, 'batch_size': 64}
Epoch 32: early stopping
Restoring model weights from the end of the best epoch: 22.
Validation loss: 0.0049
  Evaluating combination 3/180: {'units': 32, 'dropout': 0.0, 'learning_rate': 0.01, 'batch_size': 128}
Epoch 28: early stopping
Restoring model weights from the end of the best epoch: 18.
Validation loss: 0.0053
  Evaluating combination 4/180: {'units': 32, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 32}
Epoch 11: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation loss: 0.0142
  Evaluating combination 5/180: {'units': 32, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 64}
Epoch 33: early stopping
Restoring model weights from the end of the best epoch: 23.
Validation loss: 0.0053
  Evaluating combination 6/180: {'units':

2024-10-20 08:47:18.156298: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_19', 28 bytes spill stores, 28 bytes spill loads



Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation loss: nan
  Evaluating combination 94/180: {'units': 128, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 32}
Epoch 20: early stopping
Restoring model weights from the end of the best epoch: 10.
Validation loss: 0.0054
  Evaluating combination 95/180: {'units': 128, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 64}
Epoch 11: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation loss: 0.0152
  Evaluating combination 96/180: {'units': 128, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 128}
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation loss: nan
  Evaluating combination 97/180: {'units': 128, 'dropout': 0.0, 'learning_rate': 0.0005, 'batch_size': 32}
Epoch 19: early stopping
Restoring model weights from the end of the best epoch: 9.
Validation loss: 0.0053
  Evaluating combination 98/180: {'unit

2024-10-20 10:37:33.788415: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_19', 212 bytes spill stores, 316 bytes spill loads

2024-10-20 10:37:33.849223: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_19', 152 bytes spill stores, 248 bytes spill loads



Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation loss: nan
  Evaluating combination 139/180: {'units': 256, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 32}
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation loss: nan
  Evaluating combination 140/180: {'units': 256, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 64}
Epoch 22: early stopping
Restoring model weights from the end of the best epoch: 12.
Validation loss: 0.0053
  Evaluating combination 141/180: {'units': 256, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 128}
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation loss: nan
  Evaluating combination 142/180: {'units': 256, 'dropout': 0.0, 'learning_rate': 0.0005, 'batch_size': 32}
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation loss: nan
  Evaluating combination 143/180: {'units

Completed grid search for input length: 192
  Best Validation MSE: 0.0040
  Best Hyperparameters: {'units': 32, 'dropout': 0.2, 'learning_rate': 0.01, 'batch_size': 32}

Completed grid search for input length: 216
  Best Validation MSE: 0.0035
  Best Hyperparameters: {'units': 32, 'dropout': 0.2, 'learning_rate': 0.01, 'batch_size': 128}

Completed grid search for input length: 240
  Best Validation MSE: 0.0036
  Best Hyperparameters: {'units': 32, 'dropout': 0.0, 'learning_rate': 0.01, 'batch_size': 128}

Completed grid search for input length: 264
  Best Validation MSE: 0.0038
  Best Hyperparameters: {'units': 256, 'dropout': 0.3, 'learning_rate': 0.001, 'batch_size': 32}


Completed grid search for input length: 288
  Best Validation MSE: 0.0036
  Best Hyperparameters: {'units': 256, 'dropout': 0.0, 'learning_rate': 0.01, 'batch_size': 128}

Completed grid search for input length: 312
  Best Validation MSE: 0.0035
  Best Hyperparameters: {'units': 128, 'dropout': 0.2, 'learning_rate': 0.01, 'batch_size': 128}

Completed grid search for input length: 336
  Best Validation MSE: 0.0035
  Best Hyperparameters: {'units': 64, 'dropout': 0.2, 'learning_rate': 0.01, 'batch_size': 64}

Completed grid search for input length: 360
  Best Validation MSE: 0.0036
  Best Hyperparameters: {'units': 256, 'dropout': 0.0, 'learning_rate': 0.01, 'batch_size': 32}

Completed grid search for input length: 384
  Best Validation MSE: 0.0035
  Best Hyperparameters: {'units': 32, 'dropout': 0.0, 'learning_rate': 0.01, 'batch_size': 32}


Completed grid search for input length: 408
  Best Validation MSE: 0.0036
  Best Hyperparameters: {'units': 256, 'dropout': 0.4, 'learning_rate': 0.01, 'batch_size': 64}

Completed grid search for input length: 432
  Best Validation MSE: 0.0036
  Best Hyperparameters: {'units': 64, 'dropout': 0.1, 'learning_rate': 0.01, 'batch_size': 64}


Completed grid search for input length: 456
  Best Validation MSE: 0.0039
  Best Hyperparameters: {'units': 64, 'dropout': 0.3, 'learning_rate': 0.01, 'batch_size': 64}

Completed grid search for input length: 480
  Best Validation MSE: 0.0043
  Best Hyperparameters: {'units': 64, 'dropout': 0.3, 'learning_rate': 0.01, 'batch_size': 64}

Completed grid search for input length: 504
  Best Validation MSE: 0.0048
  Best Hyperparameters: {'units': 128, 'dropout': 0.3, 'learning_rate': 0.01, 'batch_size': 64}

## 8. Storing Results

In [19]:
# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Expand the hyperparameters dictionary into separate columns for clarity
hyperparams_df = results_df['Best_Hyperparameters'].apply(pd.Series)

# Combine the main dataframe with hyperparameters
final_results_df = pd.concat([results_df.drop('Best_Hyperparameters', axis=1), hyperparams_df], axis=1)

# Display the final dataframe
print("Final Results DataFrame:")
final_results_df

Final Results DataFrame:


,Input_Length,Best_MSE,Validation_MSE,Validation_MAE,Validation_MAPE,Validation_RMSE,units,dropout,learning_rate,batch_size
0,360,0.003563,0.003563,0.042866,40.809849,0.059695,256.0,0.0,0.01,32.0
1,384,0.003470,0.003470,0.042129,38.756887,0.058907,32.0,0.0,0.01,32.0
2,408,0.003573,0.003573,0.043253,50.933360,0.059774,256.0,0.4,0.01,64.0
3,432,0.003650,0.003649,0.042154,41.099230,0.060411,64.0,0.1,0.01,64.0
4,456,0.003933,0.003933,0.044870,46.098188,0.062716,64.0,0.3,0.01,64.0
5,480,0.004323,0.004323,0.046985,47.611384,0.065748,64.0,0.3,0.01,64.0
6,504,0.004754,0.004754,0.049175,51.245178,0.068949,128.0,0.3,0.01,64.0


## Grid Search Results

| Input_Length | Best Validation MSE | units | dropout | learning_rate | batch_size |
|--------------|---------------------|-------|---------|---------------|------------|
| 24           | 0.004132            | 256   | 0.4     | 0.001         | 32         |
| 48           | 0.004258            | 32    | 0.1     | 0.010         | 64         |
| 72           | 0.004136            | 64    | 0.1     | 0.010         | 64         |
| 96           | 0.004128            | 64    | 0.1     | 0.010         | 64         |
| 120          | 0.003844            | 128   | 0.2     | 0.010         | 64         |
| 144          | 0.003995            | 32    | 0.0     | 0.010         | 64         |
| 168          | 0.004008            | 32    | 0.0     | 0.001         | 32         |
| 192          | 0.0040              | 32    | 0.2     | 0.01          | 32         |
| 216          | 0.0035              | 32    | 0.2     | 0.01          | 128        |
| 240          | 0.0036              | 32    | 0.0     | 0.01          | 128        |
| 264          | 0.0038              | 256   | 0.3     | 0.001         | 32         |
| 288          | 0.0036              | 256   | 0.0     | 0.01          | 128        |
| 312          | 0.0035              | 128   | 0.2     | 0.01          | 128        |
| 336          | 0.0035              | 64    | 0.2     | 0.01          | 64         |
| 360          | 0.0036              | 256   | 0.0     | 0.01          | 32         |
| 384          | 0.0035              | 32    | 0.0     | 0.01          | 32         |
| 408          | 0.0036              | 256   | 0.4     | 0.01          | 64         |
| 432          | 0.0036              | 64    | 0.1     | 0.01          | 64         |
| 456          | 0.0039              | 64    | 0.3     | 0.01          | 64         |
| 480          | 0.0043              | 64    | 0.3     | 0.01          | 64         |
| 504          | 0.0048              | 128   | 0.3     | 0.01          | 64         |


## 9. Retrain the model after getting the best hyperparameters of each input length

In [18]:
best_hyperparameters = {
    24: {
        'units': 256,
        'dropout': 0.4,
        'learning_rate': 0.001,
        'batch_size': 32
    },
    48: {
        'units': 32,
        'dropout': 0.1,
        'learning_rate': 0.01,
        'batch_size': 64
    },
    72: {
        'units': 64.0,
        'dropout': 0.1,
        'learning_rate': 0.01,
        'batch_size': 64.0
    },
    96: {
        'units': 64,
        'dropout': 0.1,
        'learning_rate': 0.01,
        'batch_size': 64
    },
    120: {
        'units': 128,
        'dropout': 0.2,
        'learning_rate': 0.01,
        'batch_size': 64
    },
    144: {
        'units': 32,
        'dropout': 0.0,
        'learning_rate': 0.01,
        'batch_size': 64
    },
    168: {
        'units': 32,
        'dropout': 0.0,
        'learning_rate': 0.001,
        'batch_size': 32
    },
    192: {
        'units': 32,
        'dropout': 0.2,
        'learning_rate': 0.01,
        'batch_size': 32
    },
    216: {
        'units': 32,
        'dropout': 0.2,
        'learning_rate': 0.01,
        'batch_size': 128
    },
    240: {
        'units': 32,
        'dropout': 0.0,
        'learning_rate': 0.01,
        'batch_size': 128
    },
    264: {
        'units': 256,
        'dropout': 0.3,
        'learning_rate': 0.001,
        'batch_size': 32
    },
    288: {
        'units': 256,
        'dropout': 0.0,
        'learning_rate': 0.01,
        'batch_size': 128
    },
    312: {
        'units': 128,
        'dropout': 0.2,
        'learning_rate': 0.01,
        'batch_size': 128
    },
    336: {
        'units': 64,
        'dropout': 0.2,
        'learning_rate': 0.01,
        'batch_size': 64
    },
    360: {
        'units': 256,
        'dropout': 0.0,
        'learning_rate': 0.01,
        'batch_size': 32
    },
    384: {
        'units': 32,
        'dropout': 0.0,
        'learning_rate': 0.01,
        'batch_size': 32
    },
    408: {
        'units': 256,
        'dropout': 0.4,
        'learning_rate': 0.01,
        'batch_size': 64
    },
    432: {
        'units': 64,
        'dropout': 0.1,
        'learning_rate': 0.01,
        'batch_size': 64
    },
    456: {
        'units': 64,
        'dropout': 0.3,
        'learning_rate': 0.01,
        'batch_size': 64
    },
    480: {
        'units': 64,
        'dropout': 0.3,
        'learning_rate': 0.01,
        'batch_size': 64
    },
    504: {
        'units': 128,
        'dropout': 0.3,
        'learning_rate': 0.01,
        'batch_size': 64
    }
}


In [19]:
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

In [20]:
def train_model(hyperparams,data_dict,length, seed=None):  # add seed
    if seed is not None:
        set_seed(seed)
    
    #get the data of each length
    X_train = data_dict[length]['X_train']
    y_train = data_dict[length]['y_train']
    X_val = data_dict[length]['X_val']
    y_val = data_dict[length]['y_val']
    
    # Train the model
    model = build_lstm_model(hyperparams, length)    
    # Early Stopping Callback
    early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True,verbose=1)    
    
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=hyperparams['batch_size'],
        validation_data=(X_val, y_val),
        callbacks=[early_stop],
        verbose=0  # Set to 1 to see training progress
    )
    
    # Retrieve the best validation MSE from the history
    best_mse = min(history.history['val_loss'])
    print(f"Validation loss: {best_mse:.5f}")
    
    return model, best_mse

In [21]:
# Make predictions
def make_prediction(model, X_obs, y_obs):
    y_pred = model.predict(X_obs,verbose=0)
    n_samples = X_obs.shape[0]
    output_len = y_obs.shape[1]

    # Reshape for inverse scaling
    y_pred_reshaped = y_pred.reshape(-1, 1)
    y_obs_reshaped = y_obs.reshape(-1, 1)

    # Inverse transform
    y_pred_inverse = scaler.inverse_transform(y_pred_reshaped).reshape(n_samples, output_len)
    y_obs_inverse = scaler.inverse_transform(y_obs_reshaped).reshape(n_samples, output_len)

    return y_pred_inverse, y_obs_inverse

In [22]:
# Compute Metrics for Each Time Step
def evaluation(y_pred_inverse, y_obs_inverse):
    
    output_len = y_pred_inverse.shape[1]
    metrics_list = []  # To store metrics for each time step
    
    for i in range(output_len):
        y_true = y_obs_inverse[:, i]
        y_pred = y_pred_inverse[:, i]

        # Mean Absolute Error (MAE)
        mae = mean_absolute_error(y_true, y_pred)

        # Mean Squared Error (MSE)
        mse = mean_squared_error(y_true, y_pred)

        # Root Mean Squared Error (RMSE)
        rmse = np.sqrt(mse)

        # Mean Absolute Percentage Error (MAPE)
        # Avoid division by zero by adding a small epsilon to y_test_flat if necessary
        epsilon = 1e-10
        y_true_safe = np.where(y_true == 0, epsilon, y_true)
        mape = np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

                # Append the metrics for the current time step to the list
        metrics_list.append({
            'Time Step': i + 1,
            'MAE': mae,
            'RMSE': rmse,
            'MAPE (%)': mape
        })

    # Create a DataFrame from the list of metrics
    metrics_df = pd.DataFrame(metrics_list)
    metrics_df.set_index('Time Step', inplace=True)

    return metrics_df

In [27]:
mean_metrics_list=[]
for length in best_hyperparameters.keys():
    print(length)
    
    # Number of runs
    n_runs = 10
    
    # get the best hyperparameter of each length
    hyperparams = best_hyperparameters[length]
    # Initialize lists to store mean metrics and all metrics from each run
    mean_metrics_rows = []
    df_all_metrics_list = []

    for run in range(1, n_runs + 1):
        print(f"\n--- Run {run} ---")

        # Optionally set a unique seed for each run to ensure variability
        seed = run  
        # Train the model
        model, mse = train_model(hyperparams, data_dict, length, seed=seed)
        
        X_train = data_dict[length]['X_train']
        y_train = data_dict[length]['y_train']
        X_val = data_dict[length]['X_val']
        y_val = data_dict[length]['y_val']
        X_test = data_dict[length]['X_test']
        y_test = data_dict[length]['y_test']

        #get the true flow and predicted flow
        y_pred_train, y_obs_train = make_prediction(model, X_train, y_train)
        y_pred_val, y_obs_val = make_prediction(model, X_val, y_val)
        y_pred_test, y_obs_test = make_prediction(model, X_test, y_test)

        #calculate the evaluation metrics of each output step
        df_train = evaluation(y_pred_train, y_obs_train).add_suffix('_train')
        df_val = evaluation(y_pred_val, y_obs_val).add_suffix('_val')
        df_test = evaluation(y_pred_test, y_obs_test).add_suffix('_test')

        df_all_metrics = pd.concat([df_train, df_val, df_test], axis=1)
        df_all_metrics.index.name = length
        
        # Append df_all_metrics to the list
        df_all_metrics_list.append(df_all_metrics)

        # Calculate mean for all output step
        mean_metrics = df_val.mean()
        mean_metrics_row = pd.DataFrame(mean_metrics).T
        mean_metrics_row['MSE_val(loss)'] = mse
        mean_metrics_row['input_len'] = length
        mean_metrics_row = mean_metrics_row[['input_len','MSE_val(loss)', 'MAE_val', 'RMSE_val', 'MAPE (%)_val']]
        
        # Append to the list
        mean_metrics_rows.append(mean_metrics_row)
        
    # Concatenate all df_all_metrics into a single DataFrame with a new level for runs
    concatenated_all_metrics = pd.concat(df_all_metrics_list, keys=range(1, n_runs + 1), names=['Run', 'Time Step'])

    # Calculate the mean across runs for each metric and time step
    # This will group by 'Time Step' and calculate the mean of each metric across all runs
    aggregated_all_metrics_mean = concatenated_all_metrics.groupby('Time Step').mean()
    aggregated_all_metrics_mean.index.name = length

    print("\n--- Aggregated Mean of All Metrics Across 10 Runs ---")
    display(aggregated_all_metrics_mean)

    # After all runs, create a DataFrame of mean metrics
    mean_metrics_df = pd.concat(mean_metrics_rows, ignore_index=True)
    # Calculate the mean of each metric across the 10 runs
    final_mean_metrics = mean_metrics_df.mean()

    # Create a DataFrame for the final mean metrics
    final_mean_metrics_df = pd.DataFrame(final_mean_metrics).T
    
    mean_metrics_list.append(final_mean_metrics_df)

mean_metrics_df = pd.concat(mean_metrics_list).reset_index(drop=True)
print("\n--- Final Mean Metrics Across 10 Runs ---")
mean_metrics_df

24

--- Run 1 ---


E:\program\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 53: early stopping
Restoring model weights from the end of the best epoch: 33.
Validation loss: 0.00422

--- Run 2 ---


E:\program\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 37: early stopping
Restoring model weights from the end of the best epoch: 17.
Validation loss: 0.00439

--- Run 3 ---


E:\program\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


KeyboardInterrupt: 

# Code to predict next 6 steps simultaneously

#### We will use
* the last 12 steps

* previous one week (24 steps)

* previous one month  (168 steps)

*  to forecast current (0 step) and 5 steps ahead

In [11]:
# Create input-output sequences with the provided function
X_train, y_train, X_train_df, y_train_df = create_multi_step_sequence(train_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=6)
X_valid, y_valid, X_valid_df, y_valid_df = create_multi_step_sequence(valid_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=6)
X_test, y_test, X_test_df, y_test_df = create_multi_step_sequence(test_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=6)

ValueError: cannot reshape array of size 1141040 into shape (168,1)

## 5. Define LSTM model

In [12]:
def create_lstm_model_multi_step(input_shape, n_outputs, units, dropout_rate, learning_rate):
    model = keras.Sequential()
    model.add(LSTM(units=units, activation='tanh', input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(n_outputs))
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        loss='mse',  # Mean Squared Error loss for regression
        optimizer=optimizer,
        metrics=['mae']  # Mean Absolute Error as a metric
    )
    return model

input_shape = (x_train_scaled.shape[1], x_train_scaled.shape[2])
n_outputs = y_train.shape[1]

## 6. Set Up Hyperparameter Grid

In [17]:
# Hyperparameter options
units_list = [50, 100, 200]
dropout_rates = [0, 0.3, 0.5]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [32, 64, 128]

# Create all possible combinations
hyperparameter_combinations = list(product(units_list, dropout_rates, learning_rates, batch_sizes))

## 7. Train the Model with Hyperparameter Tuning

In [14]:
# Initialize variables to store the best model and hyperparameters
best_val_mae = np.inf
best_hyperparams = None
best_model = None

for idx, (units, dropout_rate, learning_rate, batch_size) in enumerate(hyperparameter_combinations):
    print(f"\nCombination {idx+1}/{len(hyperparameter_combinations)}")
    print(f"Training with units={units}, dropout_rate={dropout_rate}, learning_rate={learning_rate}, batch_size={batch_size}")

    # Create the LSTM model with the current hyperparameters
    model = create_lstm_model_multi_step(
        input_shape=input_shape,
        n_outputs=n_outputs,
        units=units,
        dropout_rate=dropout_rate,
        learning_rate=learning_rate
    )

    # Initialize EarlyStopping
    early_stopping = EarlyStopping(
        monitor='val_mae',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

    # Train the model
    history = model.fit(
        x_train_scaled, y_train_scaled,
        epochs=50,
        batch_size=batch_size,
        validation_data=(x_val_scaled, y_val_scaled),
        callbacks=[early_stopping],
        verbose=0
    )

    # Get the best validation MAE from this training run
    val_mae = min(history.history['val_mae'])
    print(f"Validation MAE: {val_mae:.4f}")

    # Update best model if current one is better
    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_model = model
        best_hyperparams = {
            'units': units,
            'dropout_rate': dropout_rate,
            'learning_rate': learning_rate,
            'batch_size': batch_size
        }

print("\nBest Hyperparameters:")
for param, value in best_hyperparams.items():
    print(f"{param}: {value}")
print(f"Best Validation MAE: {best_val_mae:.4f}")


Combination 1/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=32


E:\program\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 43: early stopping
Restoring model weights from the end of the best epoch: 33.
Validation MAE: 0.0602

Combination 2/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=64
Epoch 35: early stopping
Restoring model weights from the end of the best epoch: 25.
Validation MAE: 0.0600

Combination 3/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=128
Epoch 35: early stopping
Restoring model weights from the end of the best epoch: 25.
Validation MAE: 0.0600

Combination 4/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=32
Restoring model weights from the end of the best epoch: 50.
Validation MAE: 0.0618

Combination 5/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=64
Restoring model weights from the end of the best epoch: 44.
Validation MAE: 0.0634

Combination 6/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=128
Restoring model weights from the end of the b

Restoring model weights from the end of the best epoch: 50.
Validation MAE: 0.0708

Combination 46/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=32
Epoch 27: early stopping
Restoring model weights from the end of the best epoch: 17.
Validation MAE: 0.0605

Combination 47/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=64
Epoch 31: early stopping
Restoring model weights from the end of the best epoch: 21.
Validation MAE: 0.0606

Combination 48/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=128
Epoch 37: early stopping
Restoring model weights from the end of the best epoch: 27.
Validation MAE: 0.0608

Combination 49/81
Training with units=100, dropout_rate=0.5, learning_rate=0.001, batch_size=32
Restoring model weights from the end of the best epoch: 50.
Validation MAE: 0.0616

Combination 50/81
Training with units=100, dropout_rate=0.5, learning_rate=0.001, batch_size=64
Restoring model weights fro

In [ ]:
13:33

Best Hyperparameters for full time:
units: 200
dropout_rate: 0.5
learning_rate: 0.01
batch_size: 32
Best Validation MAE: 0.0503

Best Hyperparameters for after covid:
units: 100
dropout_rate: 0.3
learning_rate: 0.01
batch_size: 128
Best Validation MAE: 0.0595

## 8. Make predictions

In [15]:
# Make predictions
y_pred_scaled = best_model.predict(x_test_scaled)

# Reshape for inverse scaling
y_pred_reshaped = y_pred_scaled.reshape(-1, 1)
y_test_reshaped = y_test_scaled.reshape(-1, 1)

# Inverse transform
y_pred_inverse = y_scaler.inverse_transform(y_pred_reshaped).reshape(n_test_samples, n_outputs)
y_test_inverse = y_scaler.inverse_transform(y_test_reshaped).reshape(n_test_samples, n_outputs)

y_test_flat = y_test_inverse.flatten()
y_pred_flat = y_pred_inverse.flatten()

66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


## 9. Evaluating the LSTM Model

In [16]:
# Compute Metrics for Each Time Step

for i in range(n_outputs):
    y_true = y_test_inverse[:, i]
    y_pred = y_pred_inverse[:, i]

    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_true, y_pred)

    # Mean Squared Error (MSE)
    mse = mean_squared_error(y_true, y_pred)

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)

    # Mean Absolute Percentage Error (MAPE)
    # Avoid division by zero by adding a small epsilon to y_test_flat if necessary
    epsilon = 1e-10
    y_true_safe = np.where(y_true == 0, epsilon, y_true)
    mape = np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

    print(f"\nTime Step {i+1} Evaluation Metrics:")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MAPE: {mape:.2f}%")


Time Step 1 Evaluation Metrics:
RMSE: 28.4454
MAE: 20.8450
MAPE: 19.44%

Time Step 2 Evaluation Metrics:
RMSE: 36.2563
MAE: 26.7673
MAPE: 25.02%

Time Step 3 Evaluation Metrics:
RMSE: 43.7203
MAE: 33.1341
MAPE: 29.97%

Time Step 4 Evaluation Metrics:
RMSE: 49.3573
MAE: 38.0972
MAPE: 35.19%

Time Step 5 Evaluation Metrics:
RMSE: 53.2812
MAE: 41.5888
MAPE: 39.42%

Time Step 6 Evaluation Metrics:
RMSE: 53.3926
MAE: 42.0135
MAPE: 44.19%


# Code to predict next 6 steps step-by-step

#### We will use
* the last 12 steps

* previous one week (24 steps)

* previous one month  (168 steps)

*  to forecast current (0 step)

## 4. Create input and output data

In [29]:
# Create input-output sequences with the provided function
X_train, y_train, X_train_df, y_train_df = create_multi_step_sequence(train_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=1)
X_valid, y_valid, X_valid_df, y_valid_df = create_multi_step_sequence(valid_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=1)
X_test, y_test, X_test_df, y_test_df = create_multi_step_sequence(test_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=1)

In [42]:
X_train.shape, X_valid.shape, X_test.shape

((19570, 14, 1), (6907, 14, 1), (5623, 14, 1))

## 5. Normalise the data after split (step-by-step)

Normalise X

In [31]:
# Separate scalers for inputs and outputs
x_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))

# Reshape x_train to 2D for scaling
n_samples, n_timesteps, n_features = X_train.shape
x_train_reshaped = X_train.reshape(-1, n_features)  # Shape: (n_samples * n_timesteps, n_features)
# Fit the scaler on the training data
x_scaler.fit(x_train_reshaped)
# Transform the training data
x_train_scaled = x_scaler.transform(x_train_reshaped)
# Reshape back to original shape
x_train_scaled = x_train_scaled.reshape(n_samples, n_timesteps, n_features)

# x_val
n_val_samples = X_valid.shape[0]
x_val_reshaped = X_valid.reshape(-1, n_features)
x_val_scaled = x_scaler.transform(x_val_reshaped)
x_val_scaled = x_val_scaled.reshape(n_val_samples, n_timesteps, n_features)

# x_test
n_test_samples = X_test.shape[0]
x_test_reshaped = X_test.reshape(-1, n_features)
x_test_scaled = x_scaler.transform(x_test_reshaped)
x_test_scaled = x_test_scaled.reshape(n_test_samples, n_timesteps, n_features)

Normalise y

In [32]:
# Reshape y_train to 2D for scaling
y_train_reshaped = y_train.reshape(-1, 1)  # Shape: (n_samples * n_outputs, 1)
# Fit the scaler on the training data
y_scaler.fit(y_train_reshaped)
# Transform the training data
y_train_scaled = y_scaler.transform(y_train_reshaped)
# Reshape back to original shape
y_train_scaled = y_train_scaled.reshape(n_samples, y_train.shape[1])

# y_val
y_val_reshaped = y_valid.reshape(-1, 1)
y_val_scaled = y_scaler.transform(y_val_reshaped)
y_val_scaled = y_val_scaled.reshape(n_val_samples, y_valid.shape[1])

# y_test
y_test_reshaped = y_test.reshape(-1, 1)
y_test_scaled = y_scaler.transform(y_test_reshaped)
y_test_scaled = y_test_scaled.reshape(n_test_samples, y_test.shape[1])

## 6. Build LSTM

In [21]:
def create_lstm_model(input_shape, units, dropout_rate, learning_rate):
    model = keras.Sequential()
    model.add(LSTM(units=units, activation='tanh', input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1)) # Output layer for one-step prediction
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        loss='mse',
        optimizer=optimizer,
        metrics=['mae']
    )
    return model

input_shape = (x_train_scaled.shape[1], x_train_scaled.shape[2])

## 7. Hyperparameter Tuning

In [22]:
# Hyperparameter options
units_list = [50, 100, 200]
dropout_rates = [0, 0.3, 0.5]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [32, 64, 128]

# Create all possible combinations
hyperparameter_combinations = list(product(units_list, dropout_rates, learning_rates, batch_sizes))

In [23]:
# Initialize variables to store the best model and hyperparameters
best_val_mae = np.inf
best_hyperparams = None
best_model = None

for idx, (units, dropout_rate, learning_rate, batch_size) in enumerate(hyperparameter_combinations):
    print(f"\nCombination {idx+1}/{len(hyperparameter_combinations)}")
    print(f"Training with units={units}, dropout_rate={dropout_rate}, learning_rate={learning_rate}, batch_size={batch_size}")

    # Create the LSTM model with the current hyperparameters
    model = create_lstm_model(
        input_shape=input_shape,
        units=units,
        dropout_rate=dropout_rate,
        learning_rate=learning_rate
    )

    # Initialize EarlyStopping
    early_stopping = EarlyStopping(
        monitor='val_mae',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

    # Train the model
    history = model.fit(
        x_train_scaled, y_train_scaled,
        epochs=50,
        batch_size=batch_size,
        validation_data=(x_val_scaled, y_val_scaled),
        callbacks=[early_stopping],
        verbose=0
    )

    # Get the best validation MAE from this training run
    val_mae = min(history.history['val_mae'])
    print(f"Validation MAE: {val_mae:.4f}")

    # Update best model if current one is better
    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_model = model
        best_hyperparams = {
            'units': units,
            'dropout_rate': dropout_rate,
            'learning_rate': learning_rate,
            'batch_size': batch_size
        }

print("\nBest Hyperparameters:")
for param, value in best_hyperparams.items():
    print(f"{param}: {value}")
print(f"Best Validation MAE: {best_val_mae:.4f}")



Combination 1/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=32


E:\program\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 40: early stopping
Restoring model weights from the end of the best epoch: 30.
Validation MAE: 0.0452

Combination 2/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=64
Epoch 44: early stopping
Restoring model weights from the end of the best epoch: 34.
Validation MAE: 0.0452

Combination 3/81
Training with units=50, dropout_rate=0, learning_rate=0.01, batch_size=128
Epoch 23: early stopping
Restoring model weights from the end of the best epoch: 13.
Validation MAE: 0.0466

Combination 4/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=32
Restoring model weights from the end of the best epoch: 48.
Validation MAE: 0.0450

Combination 5/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=64
Restoring model weights from the end of the best epoch: 49.
Validation MAE: 0.0463

Combination 6/81
Training with units=50, dropout_rate=0, learning_rate=0.001, batch_size=128
Restoring model weights from the end of the b

Restoring model weights from the end of the best epoch: 46.
Validation MAE: 0.0513

Combination 46/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=32
Epoch 29: early stopping
Restoring model weights from the end of the best epoch: 19.
Validation MAE: 0.0466

Combination 47/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=64
Epoch 25: early stopping
Restoring model weights from the end of the best epoch: 15.
Validation MAE: 0.0460

Combination 48/81
Training with units=100, dropout_rate=0.5, learning_rate=0.01, batch_size=128
Epoch 43: early stopping
Restoring model weights from the end of the best epoch: 33.
Validation MAE: 0.0458

Combination 49/81
Training with units=100, dropout_rate=0.5, learning_rate=0.001, batch_size=32
Restoring model weights from the end of the best epoch: 49.
Validation MAE: 0.0463

Combination 50/81
Training with units=100, dropout_rate=0.5, learning_rate=0.001, batch_size=64
Epoch 34: early stopping
Re

Best Hyperparameters full time:
units: 50
dropout_rate: 0
learning_rate: 0.01
batch_size: 128
Best Validation MAE: 0.0388

Best Hyperparameters after covid:
units: 100
dropout_rate: 0
learning_rate: 0.001
batch_size: 32
Best Validation MAE: 0.0450

#### Recreate the model¶

In [33]:
def best_lstm_model(input_shape, units, dropout_rate, learning_rate):
    model = keras.Sequential()
    model.add(LSTM(units=units, activation='tanh', input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1)) # Output layer for one-step prediction
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        loss='mse',
        optimizer=optimizer,
        metrics=['mae']
    )
    return model

In [34]:
input_shape = (x_train_scaled.shape[1], x_train_scaled.shape[2])
units=50
dropout_rate=0
learning_rate=0.01
batch_size=128

#### Retrain the model

In [35]:
# Initialize EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_mae',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

In [37]:
# Train the model
history = best_model.fit(
    x_train_scaled, y_train_scaled,
    epochs=50,  # You can adjust this as needed
    batch_size=batch_size,
    validation_data=(x_val_scaled, y_val_scaled),
    callbacks=[early_stopping],
    verbose=1  # Set to 1 to see detailed training output
)

Epoch 1/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0044 - mae: 0.0474 - val_loss: 0.0039 - val_mae: 0.0405
Epoch 2/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0043 - mae: 0.0465 - val_loss: 0.0039 - val_mae: 0.0406
Epoch 3/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0043 - mae: 0.0467 - val_loss: 0.0040 - val_mae: 0.0432
Epoch 4/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0044 - mae: 0.0472 - val_loss: 0.0039 - val_mae: 0.0401
Epoch 5/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0043 - mae: 0.0469 - val_loss: 0.0040 - val_mae: 0.0430
Epoch 6/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0044 - mae: 0.0473 - val_loss: 0.0039 - val_mae: 0.0399
Epoch 7/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0043 - mae: 0.0464 - val_loss: 0.0039 - val_mae: 0.0406
Epoch 8/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0045 - mae: 0.0475 - val_loss: 0.0039 - val_mae: 0.0416
Epoch 9/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/

## 8. Recursive Forecasting with LSTM (step-by-step)

In [38]:
def recursive_forecast(model, x_test_scaled, start_index, n_steps, x_scaler, y_scaler):
    """
    Perform recursive forecasting using the trained model.

    Parameters:
    - model: Trained model
    - x_test_scaled: The scaled test input data (shape: n_samples, n_timesteps, n_features)
    - start_index: The starting index in the test data
    - n_steps: Number of future steps to predict
    - x_scaler: Scaler used for input features
    - y_scaler: Scaler used for target variable

    Returns:
    - predictions: List of predicted values (in original scale)
    """
    predictions = []
    scaled_predictions = []
    current_input = x_test_scaled[start_index].copy()  # Shape: (n_timesteps, n_features)

    for step in range(n_steps):
        # Reshape to (1, n_timesteps, n_features) for prediction
        input_seq = current_input.reshape((1, current_input.shape[0], current_input.shape[1]))

        # Predict the next time step (scaled)
        yhat_scaled = model.predict(input_seq, verbose=0)  # Shape: (1, 1)

        # Inverse transform the prediction to original scale
        yhat = y_scaler.inverse_transform(yhat_scaled)[0, 0]
        
        # Transform the prediction back to input feature scale for lag features
        yhat_for_input = x_scaler.transform(yhat.reshape(-1, 1))[0, 0]

        # Append predictions
        predictions.append(yhat)
        scaled_predictions.append(yhat_for_input)

        # Move to the next time step in x_test_scaled
        next_index = start_index + step + 1
        if next_index < len(x_test_scaled):
            # Use features from the next time step
            next_input = x_test_scaled[next_index].copy()
        else:
            # Reached the end of x_test_scaled
            break

        # Update lag features with available scaled predictions
        for lag in range(1, min(step + 1, 6) + 1):
            feature_index = 12 - lag  # lag1 is at index 11
            next_input[feature_index, 0] = scaled_predictions[-lag]

        # Keep lag24 and lag168 as they are, or update if necessary

        # Set current_input for next iteration
        current_input = next_input

    return predictions

## 9. Make step-by-step prediction

In [39]:
# Number of steps to predict
n_steps = 6

# Initialize lists to store predictions and actual values
all_predictions = []
all_actuals = []

# Ensure we have enough data for recursive predictions
n_test_samples = x_test_scaled.shape[0]

for i in range(n_test_samples - n_steps):
    # Perform recursive forecasting
    predictions = recursive_forecast(
        model=best_model,
        x_test_scaled=x_test_scaled,
        start_index=i,
        n_steps=n_steps,
        x_scaler=x_scaler,
        y_scaler=y_scaler
    )

    # Get the actual future values (in original scale)
    actual_values = y_test[i+1:i + len(predictions) + 1].flatten()

    # Store the predictions and actual values
    all_predictions.append(predictions)
    all_actuals.append(actual_values)

# Convert lists to numpy arrays
all_predictions = np.array(all_predictions)
all_actuals = np.array(all_actuals)

## 10. Evaluating the LSTM Model

In [40]:
# Compute evaluation metrics
epsilon = 1e-10  # To avoid division by zero in MAPE

for i in range(n_steps):
    y_true = all_actuals[:, i]
    y_pred = all_predictions[:, i]

    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_true, y_pred)

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    # Mean Absolute Percentage Error (MAPE)
    y_true_safe = np.where(y_true == 0, epsilon, y_true)
    mape = np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

    print(f"\nTime Step {i+1} Evaluation Metrics:")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MAPE: {mape:.2f}%")


Time Step 1 Evaluation Metrics:
RMSE: 41.4159
MAE: 29.8309
MAPE: 32.89%

Time Step 2 Evaluation Metrics:
RMSE: 44.7596
MAE: 31.9444
MAPE: 36.16%

Time Step 3 Evaluation Metrics:
RMSE: 47.2820
MAE: 33.4781
MAPE: 39.15%

Time Step 4 Evaluation Metrics:
RMSE: 49.5900
MAE: 34.8770
MAPE: 41.75%

Time Step 5 Evaluation Metrics:
RMSE: 51.5312
MAE: 35.9326
MAPE: 43.32%

Time Step 6 Evaluation Metrics:
RMSE: 53.2142
MAE: 36.6299
MAPE: 44.43%


In [ ]:
step-by-step full date